In [7]:
import torch
import torch.nn as nn # NN networks (CNN, RNN, losses)
import torch.optim as optim # Aptimizers (Adam, Adadelta, Adagrad)
import torch.nn.functional as F # Activarions func (ReLU, Sigmoid) also included in nn
from torch.utils.data import DataLoader # Dataset manager
import torchvision.datasets as datasets # Datasets
import torchvision.transforms as transforms # Transformation to datasets


In [8]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparametrs
input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.001
num_epoches = 2
batch_size = 64


In [9]:
# Load data
train_dataset = datasets.MNIST(root="dataset/", train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root="dataset/", train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)


In [10]:
from turtle import forward
from unicodedata import bidirectional


class BRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes) -> None:
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_classes = num_classes
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True,
                            bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, num_classes)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device=device)
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device=device)
        
        # Forward prop
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

        

In [11]:
# Initialize model
model = BRNN(input_size, hidden_size, num_layers, num_classes).to(device=device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [12]:
# Train model
for epoch in range(num_epoches):
    for batch_ind, (data, targets) in enumerate(train_loader):
        # Data on cude
        data = data.to(device=device).squeeze(1) # ([64, 1, 28, 28])
        targets = targets.to(device=device)
        
        # Forawrd
        scores = model(data) # Equal to model.forward(data)
        loss = criterion(scores, targets)

        # Backprop
        optimizer.zero_grad()
        loss.backward()

        # Gradient descent or adam step
        optimizer.step()


In [13]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    print(f"{float(num_correct) / float(num_samples)}")
    model.train()

In [14]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)


0.9731
0.9719
